In [1]:
import sys
import os

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim





In [2]:
import torchinfo
import pytorchcv
import torchvision
from torchvision import models
from torch.utils.data import DataLoader
from torch.nn.functional import one_hot

In [3]:
import json
import pickle
import itertools
import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8') # pretty matplotlib plots
sns.set('notebook', style='whitegrid', font_scale=1.25)

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

False
0


AssertionError: Torch not compiled with CUDA enabled

In [11]:
# device = 'cuda' # TODO change to GPU if you have one (e.g. on Colab)
device = 'cpu'
# if torch.cuda.is_available():
#     device = 'cuda'
# else:
#     device = 'cpu'

host = 'nt'

if host == 'hpc':
    DATA_DIR = os.environ.get('DATA_DIR', os.path.abspath('/cluster/tufts/cs152l3dclass/jhadid01/l3d_pn_dataset1000'))
else:
    if os.name == 'nt':
        # DATA_DIR = os.environ.get('DATA_DIR', os.path.abspath("C:\\Users\\arman\\Downloads\\L3D_Project\\l3d_pn_dataset1000"))
        DATA_DIR = os.environ.get('DATA_DIR', os.path.abspath("C:\\Users\\arman\\Downloads\\L3D_Project\\l3d_pn_dataset500"))
        # DATA_DIR = os.environ.get('DATA_DIR', os.path.abspath("C:\\Users\\arman\\Downloads\\L3D_Project\\Quotient-train\\"))
    else:
        DATA_DIR = os.environ.get('DATA_DIR', os.path.abspath('../../l3d_pn_dataset1000'))

print(DATA_DIR)


/home/joseph280996/Code/School/L3D/Project/l3d_pn_dataset1000


In [12]:
!ls $DATA_DIR/train/

False_PN  True_PN


In [13]:
%load_ext autoreload
%autoreload 2

In [14]:
# Import utils from provided local starter code files
import data_utils
import models
import train

In [15]:
def eval_acc(model, device, test_loader):
    """
    Evaluate accuracy of a model on the test loader.
    Args:
        model: PyTorch model
        device: 'cuda' or 'cpu'
        test_loader: DataLoader for test data

    Returns:
        Accuracy as a float
    """
    model.to(device)
    model.eval()
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            _, predicted = torch.max(outputs, 1)  # Get predicted class
            correct += (predicted == y).sum().item()  # Count correct predictions
    return correct / len(test_loader.dataset)

In [16]:
def plot_training_progress(best_infox):
    """
    Plot training and validation progress.
    Args:
        best_infox: Dictionary containing 'epochs', 'tr' (training), and 'va' (validation) metrics.
    """
    plt.figure(figsize=(10, 6))

    # Training metrics
    plt.plot(best_infox['epochs'], best_infox['tr']['loss'], '--', color='b', label='Training Loss')
    plt.plot(best_infox['epochs'], best_infox['tr']['err'], '-', color='b', label='Training Error')

    # Validation metrics
    plt.plot(best_infox['epochs'], best_infox['va']['xent'], '--', color='r', label='Validation Loss')
    plt.plot(best_infox['epochs'], best_infox['va']['err'], '-', color='r', label='Validation Error')

    plt.xlabel('Epochs')
    plt.ylabel('Metrics')
    plt.title('Training and Validation Progress')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Assuming the PNDataset and make_PN_data_loaders are defined as provided
root_path = r"../../l3d_pn_dataset500LP"

# MixMatch hyperparameters
mixup_alpha = 0.75
temperature = 0.5
lambda_u = 75
num_classes = 2  # Adjust to your dataset

# Define sharpening function
def sharpen(probabilities, T):
    return torch.pow(probabilities, 1 / T) / torch.sum(torch.pow(probabilities, 1 / T), dim=1, keepdim=True)

# Define mixup function
def mixup(x1, y1, x2, y2, alpha):
    lam = torch.distributions.Beta(alpha, alpha).sample().item()
    lam = max(lam, 1 - lam)
    x_mix = lam * x1 + (1 - lam) * x2
    y_mix = lam * y1 + (1 - lam) * y2
    return x_mix, y_mix

# ResNet Feature Extractor
# class ResNetFeatureExtractor(nn.Module):
#     def __init__(self, num_classes):
#         super(ResNetFeatureExtractor, self).__init__()
#         # Load pre-trained ResNet
#         resnet = models.PretrainedResNetForPN(
#             src_dataset='ImageNet1k', arch='ResNet10', n_trainable_layers=1, seed=500
#             )
#         self.feature_extractor = nn.Sequential(*list(resnet.children())[:-1])  # Remove FC
#         self.fc = nn.Linear(resnet.fc.in_features, num_classes)  # Custom FC layer

#     def forward(self, x):
#         features = self.feature_extractor(x)
#         features = features.view(features.size(0), -1)  # Flatten
#         out = self.fc(features)
#         return out

# Prepare DataLoaders
# root_path = os.path.abspath(".")  # Adjust to dataset root
tr_loader, va_loader, te_loader = data_utils.make_PN_data_loaders(
    root=root_path,
    batch_size=32,
    n_samples_per_class_trainandvalid=500
)

# Model, optimizer, and criterion
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = models.PretrainedResNetForPN(
            src_dataset='ImageNet1k', arch='ResNet10', n_trainable_layers=1, seed=500
    ).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

best_infox = {
    'epochs': [],
    'tr': {'loss': [], 'err': []},
    'va': {'xent': [], 'err': []},
}

# Define your training loop
num_epochs = 50
model = models.PretrainedResNetForPN(
            src_dataset='ImageNet1k', arch='ResNet10', n_trainable_layers=1, seed=500
    ).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    correct = 0
    total_samples = 0
    
    # Training phase
    for inputs_x, targets_x in tr_loader:
        inputs_x, targets_x = inputs_x.to(device), targets_x.to(device)
        optimizer.zero_grad()
        outputs = model(inputs_x)
        loss = criterion(outputs, targets_x)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs_x.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets_x).sum().item()
        total_samples += inputs_x.size(0)

    # Training metrics
    train_loss = total_loss / total_samples
    train_err = 1 - correct / total_samples
    best_infox['tr']['loss'].append(train_loss)
    best_infox['tr']['err'].append(train_err)

    # Validation phase
    model.eval()
    val_loss = 0
    val_correct = 0
    val_samples = 0
    with torch.no_grad():
        for inputs_val, targets_val in va_loader:
            inputs_val, targets_val = inputs_val.to(device), targets_val.to(device)
            outputs_val = model(inputs_val)
            loss_val = criterion(outputs_val, targets_val)

            val_loss += loss_val.item() * inputs_val.size(0)
            _, predicted_val = outputs_val.max(1)
            val_correct += predicted_val.eq(targets_val).sum().item()
            val_samples += inputs_val.size(0)

    # Validation metrics
    val_xent = val_loss / val_samples
    val_err = 1 - val_correct / val_samples
    best_infox['va']['xent'].append(val_xent)
    best_infox['va']['err'].append(val_err)

    # Track epochs
    best_infox['epochs'].append(epoch)

    # Logging
    print(f"Epoch {epoch}/{num_epochs}")
    print(f"  Train Loss: {train_loss:.4f}, Train Error: {train_err:.4f}")
    print(f"  Val Loss: {val_xent:.4f}, Val Error: {val_err:.4f}")



splitname   0   1
    train 100 100
    valid  25  25
     test 125 125
Setup complete. Trainable parameter count=1026 over 2 tensors in layers: output.
Setup complete. Trainable parameter count=1026 over 2 tensors in layers: output.


AttributeError: 'PretrainedResNetForPN' object has no attribute 'fc'

In [ ]:
# Plot results
plot_training_progress(best_infox)

# Evaluate on test set
test_acc = eval_acc(model, device, te_loader)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
tar_acc = {}
tar_acc[('ResNet18', 'ImageNet1k')] = eval_acc(model, device, te_loader)
print(tar_acc)